In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

from keras.layers import Dense, Input
from keras.models import Sequential
from keras import optimizers

from sympy import Symbol

import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_dark'

## Data preparing

In [2]:
X, y = make_regression(n_samples=1000, n_features=4, n_informative=3, noise=0.5)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [3]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Linear Regression

In [4]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
print(lr.intercept_.round(2), *lr.coef_.round(2))
y_pred = lr.predict(X_test_scaled)
mean_squared_error(y_test, y_pred), mean_absolute_percentage_error(y_test, y_pred)

-341.2 184.4 197.8 301.25 0.04


(0.2346687389585361, 0.025235420257515444)

## Single layer MLP using Keras

In [15]:
slp = Sequential([
    Input((4,)),
    Dense(1, activation='linear', kernel_initializer='zeros')
])
slp.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5 (20.00 B)

 Trainable params: 5 (20.00 B)

 Non-trainable params: 0 (0.00 B)

In [16]:
slp.compile(loss='mse', optimizer=optimizers.SGD(0.1), metrics=['mape'])
history = slp.fit(X_train_scaled, y_train, epochs=50, batch_size=16, validation_data=(X_test_scaled, y_test))

Epoch 1/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4479.2896 - mape: 125.7153 - val_loss: 3365.8376 - val_mape: 116.5934
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3182.1917 - mape: 168.4111 - val_loss: 2482.3914 - val_mape: 80.7754
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2410.5383 - mape: 106.5609 - val_loss: 1832.3717 - val_mape: 93.4629
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1744.0317 - mape: 110.1411 - val_loss: 1464.2343 - val_mape: 123.9613
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1344.3093 - mape: 277.7689 - val_loss: 1039.1415 - val_mape: 79.5210
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 974.7681 - mape: 84.9620 - val_loss: 784.4464 - val_mape: 57.8368
Epoch 7/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 749.6094 - mape: 80.2376 - val_loss: 592.6204 - val_mape: 55.0062
Epoch 8/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 601.1764 - mape: 104.0430 - val_loss: 459.1532 - val_mape

In [7]:
loss_data = pd.DataFrame({
    'train': history.history['loss'],
    'test': history.history['val_loss']
})

px.line(loss_data, title='Loss (MSE)').show()

mape_data = pd.DataFrame({
    'train': history.history['mape'],
    'test': history.history['val_mape']
})

px.line(mape_data, title='MAPE').show()

In [8]:
for layer in slp.layers:
    print(layer.get_weights())

[array([[184.11562  ],
       [197.54471  ],
       [300.88647  ],
       [ -0.3130937]], dtype=float32), array([-340.5272], dtype=float32)]


## MLP

In [13]:
mlp = Sequential([
    Input((4,)),
    Dense(16, activation='sigmoid'),
    # Dense(8, activation='sigmoid'),
    Dense(1, activation='linear')
])
mlp.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 16)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97 (388.00 B)

 Trainable params: 97 (388.00 B)

 Non-trainable params: 0 (0.00 B)

In [14]:
mlp.compile(loss='mse', optimizer=optimizers.SGD(0.1), metrics=['mape'])

history = mlp.fit(X_train_scaled, y_train, epochs=100, batch_size=16, validation_data=(X_test_scaled, y_test))

Epoch 1/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4660.1411 - mape: 125.9056 - val_loss: 4698.4014 - val_mape: 157.4614
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5764.8726 - mape: 190.7574 - val_loss: 5972.1948 - val_mape: 266.1251
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5032.3486 - mape: 200.8628 - val_loss: 4691.2446 - val_mape: 99.9733
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5174.7896 - mape: 158.5062 - val_loss: 5685.3110 - val_mape: 239.4639
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4496.1616 - mape: 177.5049 - val_loss: 7291.3169 - val_mape: 362.8246
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5206.5869 - mape: 285.5859 - val_loss: 5473.0815 - val_mape: 218.3549
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4678.4756 - mape: 180.6632 - val_loss: 5318.7671 - val_mape: 201.4827
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4745.3735 - mape: 296.7276 - val_loss: 